In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Load CSV as data fram
cities_df = pd.read_csv("../weatherpy/cities.csv")
cities_df.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country
0,Korla,41.7597,86.1469,54.10,32,12,5.88,CN
1,At-Bashi,41.1702,75.8106,41.14,46,18,6.24,KG
2,Ushuaia,-54.8000,-68.3000,42.80,87,75,3.44,AR
3,Mahébourg,-20.4081,57.7000,77.00,94,75,14.97,MU
4,Butaritari,3.0707,172.7902,82.96,76,88,12.37,KI


<h3>Creating Heat Map (Humidity) </h3>

In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<h3>Perfect Conditions </h3>

In [5]:
# Narrowing donw df
perf_temp_df = cities_df[cities_df['Max_Temp'] > 70].reset_index(drop = True)
perf_temp_df = perf_temp_df[perf_temp_df['Max_Temp'] <= 80].reset_index(drop = True)
perf_temp_df.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country
0,Mahébourg,-20.4081,57.7000,77.0,94,75,14.97,MU
1,Hong Kong,22.2855,114.1577,75.0,88,100,1.01,HK
2,Puerto Ayora,-0.7393,-90.3518,75.7,86,35,7.07,EC
3,Kuching,1.5500,110.3333,75.2,100,20,1.14,MY
4,Arraial do Cabo,-22.9661,-42.0278,77.0,83,40,3.44,BR


In [6]:
perf_temp_df = perf_temp_df[perf_temp_df['Wind_Speed']<10].reset_index(drop = True)
perf_temp_df = perf_temp_df[perf_temp_df['Cloudiness'] == 0].reset_index(drop = True)
perf_temp_df.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country
0,Karratha,-20.7377,116.8463,78.26,28,0,9.89,AU
1,Conde,-7.2597,-34.9075,77.00,94,0,5.75,BR
2,Maceió,-9.6658,-35.7353,75.20,94,0,4.61,BR
3,Le Vauclin,14.5452,-60.8388,77.00,69,0,3.44,MQ
4,Oussouye,12.4850,-16.5469,75.20,83,0,6.91,SN


In [7]:
len(perf_temp_df)

9

<h3>Hotels </h3>

In [8]:
perf_temp_df['Hotel_Name']= ""
perf_temp_df['Hotel_lat']= ""
perf_temp_df['Hotel_lon']= ""

In [9]:
# adding hotel name to df
for index, row in perf_temp_df.iterrows():
    # set up a parameters dictionary
    lat= str(row['Latitude'])
    lon= str(row['Longitude'])
    target_coordinates = lat, lon
    current_city = row["City"]
    
    # Had to add lat and lon individual because of constant errors 
    base_url = (f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat}%2C+{lon}")
    params = {
        "keyword": "Hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    
    try:
        print(f"first hotel for {current_city} is : {hotel_data['results'][0]['name']}")
        perf_temp_df.loc[index, 'Hotel_Name'] = hotel_data['results'][0]['name']
        perf_temp_df.loc[index, 'Hotel_lat'] = hotel_data['results'][0]['geometry']['location']['lat']
        perf_temp_df.loc[index, 'Hotel_lon'] = hotel_data['results'][0]['geometry']['location']['lng']
    except:
        print('No hotel found, skipping city')
    print("---------------------------------------------------------------------------")

first hotel for Karratha is : Karratha International Hotel
---------------------------------------------------------------------------
No hotel found, skipping city
---------------------------------------------------------------------------
first hotel for Maceió is : Meridiano Hotel
---------------------------------------------------------------------------
first hotel for Le Vauclin is : Le Cap Est Lagoon Resort & Spa
---------------------------------------------------------------------------
first hotel for Oussouye is : Campement ALULUM
---------------------------------------------------------------------------
first hotel for Boueni is : LE CHISSIOUA
---------------------------------------------------------------------------
first hotel for Cabedelo is : Hotel Almagre
---------------------------------------------------------------------------
first hotel for Baracoa is : Hotel Porto Santo
---------------------------------------------------------------------------
first hotel for A

In [10]:
perf_temp_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Hotel_Name,Hotel_lat,Hotel_lon
0,Karratha,-20.7377,116.8463,78.26,28,0,9.89,AU,Karratha International Hotel,-20.737639,116.841998
1,Conde,-7.2597,-34.9075,77.00,94,0,5.75,BR,,,
2,Maceió,-9.6658,-35.7353,75.20,94,0,4.61,BR,Meridiano Hotel,-9.667967,-35.713902
3,Le Vauclin,14.5452,-60.8388,77.00,69,0,3.44,MQ,Le Cap Est Lagoon Resort & Spa,14.588411,-60.851469
4,Oussouye,12.4850,-16.5469,75.20,83,0,6.91,SN,Campement ALULUM,12.484653,-16.547932
5,Boueni,-12.9025,45.0761,77.00,94,0,3.44,YT,LE CHISSIOUA,-12.871823,45.117945
6,Cabedelo,-6.9811,-34.8339,77.00,94,0,5.75,BR,Hotel Almagre,-7.020941,-34.830177
7,Baracoa,20.3467,-74.4958,76.98,80,0,6.71,CU,Hotel Porto Santo,20.357563,-74.505678
8,Alta Floresta,-9.8756,-56.0861,75.20,100,0,2.86,BR,Floresta Amazonica Hotel,-9.877886,-56.097149


In [11]:
# Dropping Conde because we have no hotel to stay at
perf_temp_df = perf_temp_df.drop([1]).reset_index(drop = True)

In [12]:
perf_temp_df


,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Hotel_Name,Hotel_lat,Hotel_lon
0,Karratha,-20.7377,116.8463,78.26,28,0,9.89,AU,Karratha International Hotel,-20.737639,116.841998
1,Maceió,-9.6658,-35.7353,75.20,94,0,4.61,BR,Meridiano Hotel,-9.667967,-35.713902
2,Le Vauclin,14.5452,-60.8388,77.00,69,0,3.44,MQ,Le Cap Est Lagoon Resort & Spa,14.588411,-60.851469
3,Oussouye,12.4850,-16.5469,75.20,83,0,6.91,SN,Campement ALULUM,12.484653,-16.547932
4,Boueni,-12.9025,45.0761,77.00,94,0,3.44,YT,LE CHISSIOUA,-12.871823,45.117945
5,Cabedelo,-6.9811,-34.8339,77.00,94,0,5.75,BR,Hotel Almagre,-7.020941,-34.830177
6,Baracoa,20.3467,-74.4958,76.98,80,0,6.71,CU,Hotel Porto Santo,20.357563,-74.505678
7,Alta Floresta,-9.8756,-56.0861,75.20,100,0,2.86,BR,Floresta Amazonica Hotel,-9.877886,-56.097149


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perf_temp_df.iterrows()]
locations = perf_temp_df[["Hotel_lat", "Hotel_lon"]]

In [22]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))